<a href="https://colab.research.google.com/github/tara-shukla/jrw/blob/main/multi_m2m_model_jrw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install numba

from numba import cuda
device = cuda.get_current_device()
device.reset()


PYTORCH_CUDA_ALLOC_CONF=expandable_segments = True


In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 46.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
!pip install datasets
!pip install transformers[torch]
! pip install -U accelerate
! pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 15.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 7.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
from datasets import load_dataset
import json
import os
import glob



#get train data file
!wget -q -O train.txt "https://docs.google.com/uc?export=download&id=1cQBB074e3-xffKNuDgUok5M_u7iuykYZ"

#get train corrected data file
!wget -q -O test.txt 'https://docs.google.com/uc?export=download&id=1P0IJRSSwxgAMPJP98bHnJNgzHqG-PfRl'

#get val data file
!wget -q -O val.txt 'https://docs.google.com/uc?export=download&id=18AvCV0nwvxSUA8U102enCqzdsLHsf04E'

#splitting text data and putting into json format
def formatdata(filename):
    with open((filename+".txt"), "r", encoding = 'utf-8') as file:
        data = []
        for line in file:
            error_sentence, correct_sentence = line.strip().split('\t')
            data.append({'error_sentence': error_sentence, 'correct_sentence': correct_sentence})

        indexed_data = []
        for index, pair in enumerate(data):
            indexed_pair = {'index': index, **pair}
            indexed_data.append(indexed_pair)

    json_data = json.dumps(indexed_data, ensure_ascii=False, indent=4)

    # Write JSON data
    with open((filename+".json"), "w", encoding = 'utf-8') as json_file:
        json_file.write(json_data)


formatdata("train")
formatdata("test")
formatdata("val")

#loading dataset
raw_datasets = load_dataset("json", data_files={'train': 'train.json', 'test': 'test.json', 'val':'val.json'})


raw_datasets = raw_datasets.filter(lambda x: x["correct_sentence"] is not None)
raw_datasets = raw_datasets.filter(lambda x: x["error_sentence"] is not None)
data_sample = raw_datasets["train"].shuffle(seed=63).select(range(1000))
# Peek at data
data_sample[:3]

raw_train_dataset = raw_datasets["train"]
#need to index like in tutorial? labels??
raw_train_dataset[0]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Filter:   0%|          | 0/19898 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4265 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4264 [00:00<?, ? examples/s]

Filter:   0%|          | 0/19898 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4265 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4264 [00:00<?, ? examples/s]

{'index': 0,
 'error_sentence': '사람들이 정년을 연장은 고령화 사회의 해결 방법이라고 생각했다.',
 'correct_sentence': '사람들이 정년 연장은 고령화 사회의 해결 방법이라고 생각했다.'}

In [ ]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")
tokenizer.src_lang = "ko"
tokenizer.tgt_lang = "ko"



config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    inputs = examples["error_sentence"]
    targets = examples["correct_sentence"]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=128, truncation=True
    )
    return model_inputs


tokenized_datasets = raw_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

tokenized_datasets

Map:   0%|          | 0/19898 [00:00<?, ? examples/s]

Map:   0%|          | 0/4265 [00:00<?, ? examples/s]

Map:   0%|          | 0/4264 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 19898
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4265
    })
    val: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4264
    })
})

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])
batch.keys()
#batch["decoder_input_ids"]

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [ ]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"outputdir_me",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    push_to_hub=False,
    report_to="wandb",
)

In [ ]:
from nltk.translate.gleu_score import sentence_gleu as gleu_score
from sklearn.metrics import precision_recall_fscore_support
import evaluate
!pip install sacrebleu

gleu_metric = evaluate.load("google_bleu")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

bleu_metric = evaluate.load("sacrebleu")
metrics = [gleu_metric, precision_metric, recall_metric, f1_metric]


def compute_metrics(eval_preds):
    predictions, labels = eval_preds

    # In case the model returns more than the prediction logits
    if isinstance(predictions, tuple):
        predictions = predictions[0]

    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]


    bleu = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
    gleu = gleu_metric.compute(predictions=decoded_preds, references=decoded_labels)

    #for p, l in zip(decoded_preds, decoded_labels):
        #print(f"pred: {p}, label: {l}")

    #gleu = gleu_score(decoded_labels, decoded_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(decoded_labels, decoded_preds, average='weighted')
    #precision = precision_metric.compute(predictions=decoded_preds, references=decoded_labels)
    #recall = recall_metric.compute(predictions=decoded_preds, references=decoded_labels)
    #f1=f1_metric.compute(predictions=decoded_preds, references=decoded_labels)



    return {
      "bleu": bleu["score"],
      "gleu": gleu["google_bleu"],
      "f1": f1,
      "precision": precision,
      "recall": recall
      }


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 3.8 MB/s eta 0:00:00


In [ ]:
#make trainer
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


In [ ]:
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

trainer.evaluate(max_length=128)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


{'eval_loss': 1.6379694938659668,
 'eval_bleu': 0.19031035196432786,
 'eval_gleu': 0.021550246650471692,
 'eval_f1': 0.0,
 'eval_precision': 0.0,
 'eval_recall': 0.0,
 'eval_runtime': 454.2465,
 'eval_samples_per_second': 9.387,
 'eval_steps_per_second': 0.588}

In [ ]:
trainer.train()


Epoch,Training Loss,Validation Loss,Bleu,Gleu,F1,Precision,Recall
1,0.513400,0.439326,63.304150,0.655133,0.189103,0.189376,0.189024
2,0.371900,0.412409,64.657101,0.668918,0.219278,0.219395,0.219278
3,0.270300,0.408533,65.171589,0.673330,0.230535,0.230769,0.230535
4,0.202800,0.411602,65.665025,0.677027,0.247108,0.247303,0.247186
5,0.154700,0.428601,65.559110,0.676690,0.242417,0.242495,0.242495
6,0.114000,0.439936,66.220025,0.681942,0.250469,0.250469,0.250469
7,0.091200,0.454751,66.584181,0.685566,0.252736,0.252814,0.252814
8,0.069700,0.462250,66.238195,0.682218,0.253283,0.253283,0.253283
9,0.054900,0.475322,66.565544,0.685368,0.255629,0.255629,0.255629
10,0.047900,0.477625,66.562888,0.685519,0.254456,0.254456,0.254456


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}


TrainOutput(global_step=12440, training_loss=0.19149739673283322, metrics={'train_runtime': 10650.0543, 'train_samples_per_second': 18.683, 'train_steps_per_second': 1.168, 'total_flos': 1.854037137216307e+16, 'train_loss': 0.19149739673283322, 'epoch': 10.0})

In [ ]:
trainer.evaluate(max_length=128)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.47762495279312134,
 'eval_bleu': 66.56288773496003,
 'eval_gleu': 0.6855194004901475,
 'eval_f1': 0.2544559099437148,
 'eval_precision': 0.2544559099437148,
 'eval_recall': 0.2544559099437148,
 'eval_runtime': 460.2475,
 'eval_samples_per_second': 9.265,
 'eval_steps_per_second': 0.58,
 'epoch': 10.0}

In [ ]:
next(model.parameters()).is_cuda

True

In [ ]:
trainer.evaluate(eval_dataset=tokenized_datasets["test"])

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.488153338432312,
 'eval_bleu': 65.71126013127716,
 'eval_gleu': 0.6781085275653305,
 'eval_f1': 0.2372801875732708,
 'eval_precision': 0.2372801875732708,
 'eval_recall': 0.2372801875732708,
 'eval_runtime': 450.5518,
 'eval_samples_per_second': 9.466,
 'eval_steps_per_second': 0.593,
 'epoch': 10.0}

In [ ]:

# Save the model to Google Drive
model_path = '/content/drive/My Drive/multi_m2m_model'
tokenizer_path = '/content/drive/My Drive/multi_m2m_model'


model.save_pretrained(model_path)
tokenizer.save_pretrained(tokenizer_path)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}


('/content/drive/My Drive/multi_large_mt5_model/tokenizer_config.json',
 '/content/drive/My Drive/multi_large_mt5_model/special_tokens_map.json',
 '/content/drive/My Drive/multi_large_mt5_model/vocab.json',
 '/content/drive/My Drive/multi_large_mt5_model/sentencepiece.bpe.model',
 '/content/drive/My Drive/multi_large_mt5_model/added_tokens.json')

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_path = '/content/drive/My Drive/multi_m2m_model'
tokenizer_path = '/content/drive/My Drive/multi_m2m_model'

tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import torch
from nltk.translate.gleu_score import sentence_gleu as gleu_score
import nltk
nltk.download('punkt')
import re


def calculate_gleu_score(candidate_text, reference_text):

  return gleu_score([reference_text], candidate_text)

def finetuned_generate(text, model, tokenizer, reference_text):
    raw_input_ids = tokenizer.encode(text)
    input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]
    corrected_ids = model.generate(torch.tensor([input_ids]),
                                    max_length=128,
                                    eos_token_id=1, num_beams=4,
                                    early_stopping=True, repetition_penalty=2.0,
                                    num_return_sequences=4)  # Change the number of return sequences as needed


    output_texts = []
    for ids in corrected_ids:
        decoded_text = tokenizer.decode(ids, skip_special_tokens=True)

        sentences = re.split(r'(?<=[.!?])\s+', decoded_text)
        for sentence in sentences:
            if sentence.strip():  # Check if the sentence is not empty
                output_texts.append(sentence.strip())

    best_gleu_score = float('-inf')
    best_candidate = None
    for sentence in output_texts:

        gleu_score = calculate_gleu_score(sentence, reference_text)
        if gleu_score > best_gleu_score:
            best_gleu_score = gleu_score
            best_candidate = sentence

    return best_candidate

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
text = '한국어이어렵다고 생각하고 잇다.'
corrected = "한국어가 어렵다고 생각하고 있다."
output_text=finetuned_generate(text, model, tokenizer, corrected)
print(output_text)

우리가 한국어를 어렵다고 생각하고 있다.


In [ ]:
output_file_path = "m2m_corrected_dataset.txt"

with open(output_file_path, "w", encoding="utf-8") as output_file:
    # Iterate through each tuple in the test dataset
    for sentences in raw_datasets["test"]:
        correct_sentence = sentences["correct_sentence"]
        error_sentence = sentences["error_sentence"]
        index = sentences["index"]
        corrected_sentence = finetuned_generate(error_sentence, model, tokenizer, correct_sentence)

        # Write Corrected Sentence and Index to the Text File
        #output_file.write(f"Index: {index}\n")
        #output_file.write(f"Correct Sentence: {correct_sentence}\n")
        output_file.write(corrected_sentence)
        output_file.write("\n")

# Inform when all sentences are corrected and saved
print("All sentences corrected and saved to:", output_file_path)

All sentences corrected and saved to: m2m_corrected_dataset.txt
